In [1]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import numpy as np
import copy
import hashlib
import json
import warnings
import calendar
import datetime
warnings.filterwarnings('ignore')

%matplotlib inline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler


from sklearn.metrics import accuracy_score, precision_score, \
                            recall_score, f1_score, \
                            roc_auc_score, roc_curve

from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingClassifier,
    GradientBoostingRegressor,
)
from sklearn.metrics import mean_squared_error
from sklearn.tree import (
    DecisionTreeClassifier,
    DecisionTreeRegressor,
)
from math import log2

from utils.utils import process_categorical_features


from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler


In [2]:
df = pd.read_csv(os.path.join('data', 'train.csv')) 
df1=df.copy()

In [3]:
df.corr()

,id,user_balance_usd,user_balance_coin_a,user_historical_transactions_10s,user_historical_transactions_30s,exchange_a_price,exchange_b_price,exchange_b_volatility,result
id,1.000000,-0.000087,-0.000542,0.002663,0.002178,0.000967,-0.000016,0.002983,0.000897
user_balance_usd,-0.000087,1.000000,-0.587663,-0.031081,-0.048926,0.213988,0.080039,-0.085369,-0.018770
user_balance_coin_a,-0.000542,-0.587663,1.000000,0.018207,0.027556,-0.314347,-0.116180,0.035495,0.017267
user_historical_transactions_10s,0.002663,-0.031081,0.018207,1.000000,0.526806,0.034195,0.012404,0.155497,0.354665
user_historical_transactions_30s,0.002178,-0.048926,0.027556,0.526806,1.000000,0.085027,0.035226,0.242663,0.421841
exchange_a_price,0.000967,0.213988,-0.314347,0.034195,0.085027,1.000000,0.371172,0.038767,0.089232
exchange_b_price,-0.000016,0.080039,-0.116180,0.012404,0.035226,0.371172,1.000000,0.018454,0.036113
exchange_b_volatility,0.002983,-0.085369,0.035495,0.155497,0.242663,0.038767,0.018454,1.000000,0.198650
result,0.000897,-0.018770,0.017267,0.354665,0.421841,0.089232,0.036113,0.198650,1.000000


In [4]:
def meds (df):

    cats = ['very low', 'low', 'medium', 'high','very high']
    
    dic ={}
    for s in cats :
        dic[s]=0
        med = df.exchange_b_volatility[ df['exchange_a_volatility'] == s].median()
        dic[s]=med
    avg =df.exchange_b_volatility[ df['exchange_a_volatility']!='Null'].median()
    
    return dic, avg
dic , avg =meds(df)


In [5]:
def clean_data (df):
    df1 = df.copy()
    
    
    
    df1['anonymousL'] = df1.anonymous.str.replace('\d+','')
    df1['anonymousN'] =df1.anonymous.str.replace('[aA-zZ]+','')   
    df1['anonymousN']=df1['anonymousN'].astype('int64')
    df1['anonymousL']=df1['anonymousL'].astype('category')
    
    #df1['exchange_b_volatility']=df1['exchange_b_volatility'].astype('category')
    
    
    df1['exchange_a_price'] = df1['exchange_a_price'].fillna(df.exchange_b_price)
    df1['exchange_b_price'] = df1['exchange_b_price'].fillna(df.exchange_a_price)
    
    dic ,avg = meds(df1)
    df1['exchange_a_volatility']= df1['exchange_a_volatility'].map(dic)
    df1['exchange_a_volatility']= df1['exchange_a_volatility'].astype('float64')
    
    #delete if needed
    df1['exchange_a_volatility'] = df1['exchange_a_volatility'].fillna(avg)
    df1['exchange_b_volatility'] = df1['exchange_b_volatility'].fillna(avg)
    
    df1['profit'] = abs(df1.exchange_b_price - df1.exchange_a_price)
    df1['balance_a_usd']= df1.user_balance_coin_a * df1.exchange_a_price
    df1['percentage_coin']= df1.balance_a_usd/df1.user_balance_usd
    
   
    df1["exchange_highest_price"] = (df1["exchange_a_price"] > df1["exchange_b_price"]).map({True:1, False:0})
    
    balance_coin_usd = df["user_balance_coin_a"] * df['exchange_b_price']
    df1["balance_usd_perc"] = (df1['user_balance_usd']) / (df1['user_balance_usd'] + balance_coin_usd)

    df1=df1.dropna()
    
    robustscaler = RobustScaler()
    
    prof = df1[['profit']]
    robustscaler.fit(prof)
    df1['profit'] = robustscaler.transform(prof)
    
   ##here
    bal_a = df1[['balance_a_usd']]
    robustscaler.fit(bal_a)
    df1['balance_a_usd'] = robustscaler.transform(bal_a)
    
    bal_c = df1[['user_balance_coin_a']]
    robustscaler.fit(bal_c)
    df1['user_balance_coin_a'] = robustscaler.transform(bal_c) 
    
     
    bal_u = df1[['user_balance_usd']]
    robustscaler.fit(bal_u)
    df1['user_balance_usd'] = robustscaler.transform(bal_u)
    
    per = df1[['percentage_coin']]
    robustscaler.fit(per)
    df1['percentage_coin'] = robustscaler.transform(per)
    
    
    normalizer = Normalizer(norm='l2')
    # save numerical columns to normalize
    X = df1[['exchange_a_price', 'exchange_b_price','exchange_a_volatility','exchange_b_volatility','balance_usd_perc']]

    # fit transformer to data
    normalizer.fit(X)
    X_normalized = normalizer.transform(X)
    df1['exchange_a_price'] = X_normalized[:, 0]
    df1['exchange_b_price'] = X_normalized[:, 1]
    df1['exchange_a_volatility'] = X_normalized[:, 2]
    df1['exchange_b_volatility'] = X_normalized[:, 3]
    df1['balance_usd_perc'] = X_normalized[:, 4]
    

    #nothing new 
    df_ = df1[['user_balance_usd', 'user_balance_coin_a', 'user_historical_transactions_10s', 'user_historical_transactions_30s', 'exchange_a_price', 'exchange_b_price' , 'exchange_a_volatility','exchange_b_volatility','result','profit','anonymousL','balance_usd_perc']].copy()


    return df_

In [6]:
df_clean = clean_data(df)
df_clean

,user_balance_usd,user_balance_coin_a,user_historical_transactions_10s,user_historical_transactions_30s,exchange_a_price,exchange_b_price,exchange_a_volatility,exchange_b_volatility,result,profit,anonymousL,balance_usd_perc
0,0.598907,-0.608307,0,0,0.706593,0.707546,7.545141e-06,0.000004,0,0.022358,A,0.010228
1,-0.714588,-0.064016,0,0,0.706112,0.707995,5.859371e-06,0.000012,0,0.193718,M,0.012192
2,-0.820948,2.304812,0,0,0.706572,0.707607,6.719124e-06,0.000009,0,-0.290471,F,0.006911
3,-1.075696,1.695752,0,0,0.706276,0.707926,5.089047e-06,0.000007,0,0.201621,F,0.003824
4,1.572164,-0.350857,0,0,0.707896,0.706208,4.608056e-06,0.000005,0,0.322292,A,0.012428
...,...,...,...,...,...,...,...,...,...,...,...,...
338482,-0.853421,-0.024119,0,0,0.707167,0.706980,7.915489e-06,0.000007,0,-0.625356,M,0.009710
338483,-0.283963,-0.238161,0,0,0.196266,0.980550,9.353153e-07,0.000001,0,2403.467346,A,0.001008
338484,1.584480,-0.388495,0,0,0.706352,0.707748,4.644392e-06,0.000004,0,0.133320,A,0.012635
338485,-1.141401,2.959503,0,0,0.706535,0.707668,6.241055e-06,0.000003,0,-0.212501,F,0.003650


In [7]:
'''
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.over_sampling import RandomOverSampler

df_tree= df_clean.copy()
X = df_tree.drop('result',axis=1)
y = df_tree['result']


#rus_sampler = RandomUnderSampler(sampling_strategy=.5, random_state=42)
#X_under, y_under = rus_sampler.fit_resample(X, y)

ros_sampler = RandomOverSampler(sampling_strategy=.5, random_state=42)
X_over, y_over = ros_sampler.fit_resample(X, y)

df_tree = pd.DataFrame(X_over).copy()

df_tree['result'] = y_over
'''

df_tree= df_clean.copy()



X = df_tree.drop('result',axis=1)
y = df_tree['result']
X_ = process_categorical_features(X)


X_train, X_test, y_train, y_test = train_test_split(
        X_, y, random_state=42, test_size=0.2)  







In [8]:
X_train

,user_balance_usd,user_balance_coin_a,user_historical_transactions_10s,user_historical_transactions_30s,exchange_a_price,exchange_b_price,exchange_a_volatility,exchange_b_volatility,profit,balance_usd_perc,anonymousL_A,anonymousL_F,anonymousL_M
62365,0.122884,-0.105229,0,0,0.707673,0.706497,0.000004,0.000001,0.226491,0.007776,1,0,0
190539,0.742774,0.309892,0,0,0.707086,0.707086,0.000004,0.000004,-0.728065,0.007750,1,0,0
329641,-0.626545,1.942744,0,0,0.707555,0.706605,0.000007,0.000010,-0.329594,0.008641,0,1,0
220681,-0.557184,1.130684,0,0,0.196478,0.980508,0.000002,0.000002,1403.360737,0.000777,0,1,0
317886,-0.697783,1.492397,0,0,0.707936,0.706236,0.000006,0.000009,0.080996,0.007556,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,-0.342750,0.528505,0,0,0.706580,0.707558,0.000006,0.000005,-0.225554,0.010291,0,1,0
259178,-0.574358,1.056000,0,0,0.706677,0.707496,0.000005,0.000003,-0.286927,0.007523,0,1,0
131932,-0.750648,2.084380,0,1,0.708336,0.705860,0.000008,0.000008,0.666821,0.004772,0,0,1
146867,0.400445,-0.689193,0,0,0.707997,0.706076,0.000005,0.000009,0.410073,0.014065,0,1,0


In [9]:
#log_loss , entropy,gini

#clf = RandomForestClassifier(criterion = 'log_loss', n_estimators=200 ,max_depth =20,random_state=42, min_samples_leaf =70, max_features='log2',n_jobs = -1)
# best with over spamling clf = RandomForestClassifier(criterion = 'entropy', n_estimators=300 ,max_depth =20,random_state=42, min_samples_leaf =30, max_features='log2',n_jobs = -1)

#clf = RandomForestClassifier(criterion = 'entropy', n_estimators=200 ,max_depth =18,random_state=42, min_samples_leaf =15, max_features='log2',n_jobs = -1)
clf = RandomForestClassifier(criterion = 'entropy', n_estimators=300 ,max_depth =40,random_state=42, min_samples_leaf =30, max_features='log2',n_jobs = -1)


In [ ]:
clf.fit(X_train,y_train)

predicted_proba= clf.predict_proba(X_test)
y_pred =clf.predict(X_test)

roc_auc = roc_auc_score(y_score=predicted_proba[:,1], y_true= y_test)
   
#confmat = confusion_matrix(y_true=y_test, y_pred=x1)
print('The AUROC of our classifier is = %.3f' % roc_auc)

feature_importances = pd.Series(data=clf.feature_importances_, index=X_.columns)
feature_importances.sort_values(ascending=False)




In [ ]:
#feature_importances = pd.Series(data=clf.feature_importances_, index=X_.columns)
#feature_importances.sort_values(ascending=False)
# 90 without anon N
#acrescentei balance a
#892 - acrescentar new features??
#941

In [ ]:
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot();

In [ ]:
df_t = pd.read_csv('data/test.csv', index_col='id')
df_t

In [ ]:
def clean_test (df):
    df1 = df.copy()
    
    
    
    df1['anonymousL'] = df1.anonymous.str.replace('\d','')
    df1['anonymousN'] =df1.anonymous.str.replace('[aA-zZ]+','')   
    df1['anonymousN']=df1['anonymousN'].astype('int64')
    df1['anonymousL']=df1['anonymousL'].astype('category')
    
    
    
    dic ,avg = meds(df1)
    df1['exchange_a_volatility']= df1['exchange_a_volatility'].map(dic)
    df1['exchange_a_volatility']= df1['exchange_a_volatility'].astype('float64')
    
    
    
    #delete if needed
    df1['exchange_a_volatility'] = df1['exchange_a_volatility'].fillna(avg)
    df1['exchange_b_volatility'] = df1['exchange_b_volatility'].fillna(avg)
    
    df1['exchange_a_price'] = df1['exchange_a_price'].fillna(df.exchange_b_price)
    df1['exchange_b_price'] = df1['exchange_b_price'].fillna(df.exchange_a_price)
    
    df1['profit'] = abs(df1.exchange_b_price - df1.exchange_a_price)
    
    df1['balance_a_usd']= df1.user_balance_coin_a * df1.exchange_a_price
    df1['percentage_coin']= df1.balance_a_usd/df1.user_balance_usd
    
    df1["exchange_highest_price"] = (df1["exchange_a_price"] > df1["exchange_b_price"]).map({True:1, False:0})
    
    balance_coin_usd = df["user_balance_coin_a"] * df['exchange_b_price']
    df1["balance_usd_perc"] = (df1['user_balance_usd']) / (df1['user_balance_usd'] + balance_coin_usd)
    
    df1=df1.dropna()
    
    robustscaler = RobustScaler()
    prof = df1[['profit']]
    robustscaler.fit(prof)
    df1['profit'] = robustscaler.transform(prof)
    
   ##here
    bal_a = df1[['balance_a_usd']]
    robustscaler.fit(bal_a)
    df1['balance_a_usd'] = robustscaler.transform(bal_a)
    
    bal_c = df1[['user_balance_coin_a']]
    robustscaler.fit(bal_c)
    df1['user_balance_coin_a'] = robustscaler.transform(bal_c) 
    
    
    bal_u = df1[['user_balance_usd']]
    robustscaler.fit(bal_u)
    df1['user_balance_usd'] = robustscaler.transform(bal_u)
    
    per = df1[['percentage_coin']]
    robustscaler.fit(per)
    df1['percentage_coin'] = robustscaler.transform(per)
    
    
    normalizer = Normalizer(norm='l2')
    # save numerical columns to normalize
    X = df1[['exchange_a_price', 'exchange_b_price','exchange_a_volatility','exchange_b_volatility']]

    # fit transformer to data
    normalizer.fit(X)
    X_normalized = normalizer.transform(X)
    df1['exchange_a_price'] = X_normalized[:, 0]
    df1['exchange_b_price'] = X_normalized[:, 1]
    df1['exchange_a_volatility'] = X_normalized[:, 2]
    df1['exchange_b_volatility'] = X_normalized[:, 3]
    

    
    #df1['exchange_a_volatility'] = df1['exchange_a_volatility'].astype('float')
    #df1['exchange_a_price'] = df1['exchange_a_price'].fillna(df.exchange_b_price)
    #df1['exchange_b_price'] = df1['exchange_b_price'].fillna(df.exchange_a_price)
    
    
    #df_ = df1[['user_balance_usd', 'user_balance_coin_a', 'user_historical_transactions_10s', 'user_historical_transactions_30s', 'exchange_a_price', 'exchange_b_price' , 'exchange_a_volatility','exchange_b_volatility','result','profit','anonymousN','anonymousL']].copy()
    
    #best one with random trees without hyper tunning 
    #df_ = df1[['user_balance_usd', 'user_balance_coin_a', 'user_historical_transactions_30s', 'exchange_b_price' , 'exchange_a_volatility','exchange_b_volatility','profit','exchange_a_price','user_historical_transactions_10s','anonymousN','anonymousL']].copy()
        
    #df_ = df1[['user_balance_usd', 'user_balance_coin_a', 'user_historical_transactions_30s' , 'exchange_a_volatility','exchange_b_volatility','profit','user_historical_transactions_10s','anonymousN','anonymousL']].copy()
     
    #df_ = df1[[ 'user_balance_coin_a', 'user_historical_transactions_30s' , 'exchange_a_volatility','exchange_b_volatility','profit','anonymousL']].copy()
    df_ = df1[['user_balance_usd', 'user_balance_coin_a', 'user_historical_transactions_10s', 'user_historical_transactions_30s', 'exchange_a_price', 'exchange_b_price' , 'exchange_a_volatility','exchange_b_volatility','profit','anonymousL','balance_usd_perc']].copy()

    

    
    return df_

In [ ]:
df_t_clean = clean_test(df_t)

In [ ]:
df_t_clean

In [ ]:
test_=process_categorical_features(df_t_clean)
test_

#clf.fit(X_train,y_train)

#predicted_proba= clf.predict_proba(df_t_clean)

test_=process_categorical_features(df_t_clean)

y_pred_proba = clf.predict_proba(test_)

#df_f = pd.DataFrame(y_pred_proba)
#df_f.to_csv(r'PleaseBeGood2.csv',index=True)

predicted_proba = y_pred_proba
res = pd.DataFrame(index=test_.index, data= y_pred_proba[:,1])
res['result'] = res[0]
res = res.drop(0, axis=1)

res.to_csv(r'PleaseBeGood21.csv', index = True)

In [ ]:
len(predicted_proba)

In [ ]:
test_=process_categorical_features(df_t_clean)

predicted_proba = clf.predict_proba(test_)
res = pd.DataFrame(index=test_.index, data=predicted_proba[:,1])
res['result'] = res[0]
res = res.drop(0, axis=1)

res.to_csv(r'PleaseBeGood5.csv', index = True)